# Airbnb New User Bookings

https://www.kaggle.com/c/airbnb-recruiting-new-user-bookings

Description:
"New users on Airbnb can book a place to stay in 34,000+ cities across 190+ countries. By accurately predicting where a new user will book their first travel experience, Airbnb can share more personalized content with their community, decrease the average time to first booking, and better forecast demand. In this recruiting competition, Airbnb challenges you to predict in which country a new user will make his or her first booking."

## Set Up, Initial Exploration

Imports, set options.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# enables inline plots
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.precision', 3)

**These are the files kaggle has provided us with:**

**train_users_2.csv** - the training set of users

**test_users.csv** - the test set of users

**sessions.csv** - web sessions log for users

**countries.csv** - summary statistics of destination countries in this dataset and their locations

**age_gender_bkts.csv** - summary statistics of users' age group, gender, country of destination

**sample_submission.csv** - correct format for submitting your predictions

## train_users_2.csv

**The train_users_2.csv file contains the following columns:**

id: user id

date_account_created: the date of account creation

timestamp_first_active: timestamp of the first activity, note that it can be earlier than date_account_created or date_first_booking because a user can search before signing up

date_first_booking: date of first booking

gender

age

signup_method

signup_flow: the page a user came to signup up from

language: international language preference

affiliate_channel: what kind of paid marketing

affiliate_provider: where the marketing is e.g. google, craigslist, other

first_affiliate_tracked: whats the first marketing the user interacted with before the signing up

signup_app

first_device_type

first_browser

country_destination: this is the target variable you are to predict

**Let's take a closer look at the columns.**

In [4]:
import zipfile
local_path = '/Users/eloiseheydenrych/Library/Mobile Documents/com~apple~CloudDocs'

z = zipfile.ZipFile(local_path + '/train_users_2.csv.zip')
df = pd.read_csv(z.open('train_users_2.csv'), parse_dates=[1,2])
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 213451 entries, 0 to 213450
Data columns (total 16 columns):
id                         213451 non-null object
date_account_created       213451 non-null datetime64[ns]
timestamp_first_active     213451 non-null datetime64[ns]
date_first_booking         88908 non-null object
gender                     213451 non-null object
age                        125461 non-null float64
signup_method              213451 non-null object
signup_flow                213451 non-null int64
language                   213451 non-null object
affiliate_channel          213451 non-null object
affiliate_provider         213451 non-null object
first_affiliate_tracked    207386 non-null object
signup_app                 213451 non-null object
first_device_type          213451 non-null object
first_browser              213451 non-null object
country_destination        213451 non-null object
dtypes: datetime64[ns](2), float64(1), int64(1), object(12)
memory usage: 

From the above, we can see that we have a lot of missing values in 'date_first_booking' (which makes sense because only some of the people with accounts will have made a booking). We're also missing data in 'age' and 'first_affiliate_tracked'.

Let's take a look at the first ten rows.

In [5]:
df.head(10)

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,2009-03-19 04:32:55,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,2009-05-23 17:48:09,NaN,MALE,38,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,2009-06-09 23:12:47,2010-08-02,FEMALE,56,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,2009-10-31 06:01:29,2012-09-08,FEMALE,42,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,2009-12-08 06:11:05,2010-02-18,-unknown-,41,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US
5,osr2jwljor,2010-01-01,2010-01-01 21:56:19,2010-01-02,-unknown-,NaN,basic,0,en,other,other,omg,Web,Mac Desktop,Chrome,US
6,lsw9q7uk0j,2010-01-02,2010-01-02 01:25:58,2010-01-05,FEMALE,46,basic,0,en,other,craigslist,untracked,Web,Mac Desktop,Safari,US
7,0d01nltbrs,2010-01-03,2010-01-03 19:19:05,2010-01-13,FEMALE,47,basic,0,en,direct,direct,omg,Web,Mac Desktop,Safari,US
8,a1vcnhxeij,2010-01-04,2010-01-04 00:42:11,2010-07-29,FEMALE,50,basic,0,en,other,craigslist,untracked,Web,Mac Desktop,Safari,US
9,6uh8zyj2gn,2010-01-04,2010-01-04 02:37:58,2010-01-04,-unknown-,46,basic,0,en,other,craigslist,omg,Web,Mac Desktop,Firefox,US


## test_users.csv

In [7]:
z = zipfile.ZipFile(local_path + '/test_users.csv.zip')
df_testusers = pd.read_csv(z.open('test_users.csv'),parse_dates=[1,2])
df_testusers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62096 entries, 0 to 62095
Data columns (total 15 columns):
id                         62096 non-null object
date_account_created       62096 non-null datetime64[ns]
timestamp_first_active     62096 non-null datetime64[ns]
date_first_booking         0 non-null float64
gender                     62096 non-null object
age                        33220 non-null float64
signup_method              62096 non-null object
signup_flow                62096 non-null int64
language                   62096 non-null object
affiliate_channel          62096 non-null object
affiliate_provider         62096 non-null object
first_affiliate_tracked    62076 non-null object
signup_app                 62096 non-null object
first_device_type          62096 non-null object
first_browser              62096 non-null object
dtypes: datetime64[ns](2), float64(2), int64(1), object(10)
memory usage: 7.6+ MB


In [8]:
df_testusers.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
0,5uwns89zht,2014-07-01,2014-07-01 00:00:06,NaN,FEMALE,35,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
1,jtl0dijy2j,2014-07-01,2014-07-01 00:00:51,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
2,xx0ulgorjt,2014-07-01,2014-07-01 00:01:48,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome
3,6c6puo6ix0,2014-07-01,2014-07-01 00:02:15,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Web,Windows Desktop,IE
4,czqhjk3yfe,2014-07-01,2014-07-01 00:03:05,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Safari


## sessions.csv

**The sessions.csv file contains the following columns:**

user_id: to be joined with the column 'id' in users table

action

action_type

action_detail

device_type

secs_elapsed


Let's load the sessions.csv file and take a closer look at the columns.

In [9]:
z = zipfile.ZipFile(local_path + '/sessions.csv.zip')
df_sessions = pd.read_csv(z.open('sessions.csv'))
df_sessions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10567737 entries, 0 to 10567736
Data columns (total 6 columns):
user_id          object
action           object
action_type      object
action_detail    object
device_type      object
secs_elapsed     float64
dtypes: float64(1), object(5)
memory usage: 564.4+ MB


In [10]:
df_sessions.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435


How many unique actions are there?

In [11]:
df_sessions['action'].nunique()

359

In [ ]:
# for version where I sum up actions for list of most meaningful actions

'''
# have this be number of times someone did thing, not just 0 or 1
# remove action count from before, because other category plus my list
#will give us that already


#actions to keep
action_list = [ 'ajax_refresh_subtotal',
               'create', 'notifications', 'confirm_email',
'authenticate', 'message_to_host_focus',
               'payment_methods', 'cancellation_policies',
'profile_pic', 'faq', 'apply_coupon_click', 'payout_preferences',
               'cancellation_policy_click', 'recent_reservations',
'phone_verification_number_sucessfully_submitted',
               'pay', 'transaction_history']

df_sessions.loc[~df_sessions.action.isin(action_list), "action"] = 'other'
df_sessions['action'].nunique()

df_sessions.head(10)

df_sessions.to_csv('sessions_with_other.csv')'''

In [12]:
# create dataframe that counts up the number of actions associated with each user
df_action = df_sessions.groupby(['user_id']).count()
df_action['action_user_id'] = df_action.index
df_action = df_action.set_index(np.arange(len(df_action)))
df_action.head()

,action,action_type,action_detail,device_type,secs_elapsed,action_user_id
0,39,37,37,40,39,00023iyk9l
1,63,48,48,63,62,0010k6l0om
2,90,85,85,90,89,001wyh0pz8
3,31,31,31,31,30,0028jgx1x1
4,782,712,712,789,788,002qnbzfs5


In [13]:
# ignore the other columns for now
df_action.drop(['action_type', 'action_detail', 'device_type', 'secs_elapsed'],inplace=True,axis=1)
df_action.head()

,action,action_user_id
0,39,00023iyk9l
1,63,0010k6l0om
2,90,001wyh0pz8
3,31,0028jgx1x1
4,782,002qnbzfs5


In [14]:
# put the action dataframe together with the train dataframe
merged2 = pd.merge(left=df, right=df_action, how='left', left_on='id', right_on='action_user_id')
merged2.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,action,action_user_id
0,gxn3p5htnn,2010-06-28,2009-03-19 04:32:55,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF,NaN,NaN
1,820tgsjxq7,2011-05-25,2009-05-23 17:48:09,NaN,MALE,38,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF,NaN,NaN
2,4ft3gnwmtx,2010-09-28,2009-06-09 23:12:47,2010-08-02,FEMALE,56,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US,NaN,NaN
3,bjjt8pjhuk,2011-12-05,2009-10-31 06:01:29,2012-09-08,FEMALE,42,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other,NaN,NaN
4,87mebub9p4,2010-09-14,2009-12-08 06:11:05,2010-02-18,-unknown-,41,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US,NaN,NaN


In [15]:
merged2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 213451 entries, 0 to 213450
Data columns (total 18 columns):
id                         213451 non-null object
date_account_created       213451 non-null datetime64[ns]
timestamp_first_active     213451 non-null datetime64[ns]
date_first_booking         88908 non-null object
gender                     213451 non-null object
age                        125461 non-null float64
signup_method              213451 non-null object
signup_flow                213451 non-null int64
language                   213451 non-null object
affiliate_channel          213451 non-null object
affiliate_provider         213451 non-null object
first_affiliate_tracked    207386 non-null object
signup_app                 213451 non-null object
first_device_type          213451 non-null object
first_browser              213451 non-null object
country_destination        213451 non-null object
action                     73815 non-null float64
action_user_id          

In [16]:
# delete column we don't need
del merged2['action_user_id']

#inspect column numbers a little more closely
print merged2.action.max()
print merged2.action.min()
action_median = merged2.action.median()
print action_median

2643.0
0.0
37.0


Fill action number NaNs with median.

In [17]:
#Also tried with 0s before. Not sure what to make of there not being session data for a user.
merged2['action'].fillna(action_median, inplace = True)
        
merged2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 213451 entries, 0 to 213450
Data columns (total 17 columns):
id                         213451 non-null object
date_account_created       213451 non-null datetime64[ns]
timestamp_first_active     213451 non-null datetime64[ns]
date_first_booking         88908 non-null object
gender                     213451 non-null object
age                        125461 non-null float64
signup_method              213451 non-null object
signup_flow                213451 non-null int64
language                   213451 non-null object
affiliate_channel          213451 non-null object
affiliate_provider         213451 non-null object
first_affiliate_tracked    207386 non-null object
signup_app                 213451 non-null object
first_device_type          213451 non-null object
first_browser              213451 non-null object
country_destination        213451 non-null object
action                     213451 non-null float64
dtypes: datetime64[ns](

## train_users_2 columns, imputation

In [18]:
# NEW 2/1
# How often do ages >80 book and < 1935 book? 55%
# How often do ages 18-79 book? 55%
# How often do those without an age entered book? 24%
# So, make a categorical variable called NaN age, so we can separate out people who didn't put in an age.

#X_age = df[['age', 'country_destination']]
#X_age.head()
#X_age = X_age[X_age['age'] > 80]
#X_age = X_age[X_age['age'] < 1935]
#X_age.info()
#X_age = X_age[X_age['country_destination'] == 'NDF']
#X_age.info()
#num = 923.0/2008
#print num

# create new column with all zeros
df['entered_age'] = 1 

# fill in ones in entered_age where the age column wasn't NaN
df.loc[np.isnan(df['age']), 'entered_age'] = 0

df.head(10)

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,entered_age
0,gxn3p5htnn,2010-06-28,2009-03-19 04:32:55,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF,0
1,820tgsjxq7,2011-05-25,2009-05-23 17:48:09,NaN,MALE,38,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF,1
2,4ft3gnwmtx,2010-09-28,2009-06-09 23:12:47,2010-08-02,FEMALE,56,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US,1
3,bjjt8pjhuk,2011-12-05,2009-10-31 06:01:29,2012-09-08,FEMALE,42,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other,1
4,87mebub9p4,2010-09-14,2009-12-08 06:11:05,2010-02-18,-unknown-,41,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US,1
5,osr2jwljor,2010-01-01,2010-01-01 21:56:19,2010-01-02,-unknown-,NaN,basic,0,en,other,other,omg,Web,Mac Desktop,Chrome,US,0
6,lsw9q7uk0j,2010-01-02,2010-01-02 01:25:58,2010-01-05,FEMALE,46,basic,0,en,other,craigslist,untracked,Web,Mac Desktop,Safari,US,1
7,0d01nltbrs,2010-01-03,2010-01-03 19:19:05,2010-01-13,FEMALE,47,basic,0,en,direct,direct,omg,Web,Mac Desktop,Safari,US,1
8,a1vcnhxeij,2010-01-04,2010-01-04 00:42:11,2010-07-29,FEMALE,50,basic,0,en,other,craigslist,untracked,Web,Mac Desktop,Safari,US,1
9,6uh8zyj2gn,2010-01-04,2010-01-04 02:37:58,2010-01-04,-unknown-,46,basic,0,en,other,craigslist,omg,Web,Mac Desktop,Firefox,US,1


Looks like some people put a birth year instead of an age. For those cases, compute their age and fix the value in df. Also, Airbnb requires users to be 18 or over, and it's unlikely that there are many people over 100 here. So, change the values outside of that range to the median age.

In [21]:
# consider not fixing ages. maybe people who say age is 100 are less likely to book because they're not serious?
medianage = df['age'].median()
print medianage
#when doing test version set medianage = 34

for i in range(0, len(df['age'])):
    if df.ix[i,'age'] > 1916 and df.ix[i,'age'] < 1997:
        df.ix[i,'age'] = 2015 - df.ix[i,'age']
    elif df.ix[i,'age'] >= 100 or df.ix[i,'age'] < 18:
            df.ix[i,'age'] = medianage

34.0


What are the max and min age now?

In [23]:
print df['age'].max()
print df['age'].min()
df['age'].value_counts()

99.0
18.0


34    95519
30     6124
31     6016
29     5963
28     5939
32     5855
27     5738
33     5527
26     5044
35     4860
      ...  
90       19
93       18
99       17
98       15
92       14
91       14
89       14
88       13
94       12
97       10
Name: age, dtype: int64

Impute age with median and first_affiliate_tracked with 'nothing'.

In [24]:
# fill NaN ages with median age
df['age'].fillna(medianage, inplace = True)

# check
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 213451 entries, 0 to 213450
Data columns (total 18 columns):
id                         213451 non-null object
date_account_created       213451 non-null datetime64[ns]
timestamp_first_active     213451 non-null datetime64[ns]
gender                     213451 non-null object
age                        213451 non-null float64
signup_method              213451 non-null object
signup_flow                213451 non-null int64
language                   213451 non-null object
affiliate_channel          213451 non-null object
affiliate_provider         213451 non-null object
first_affiliate_tracked    207386 non-null object
signup_app                 213451 non-null object
first_device_type          213451 non-null object
first_browser              213451 non-null object
country_destination        213451 non-null object
entered_age                213451 non-null int64
Month_Account_Created      213451 non-null int64
Month_First_Active       

Currently the dates are being stored in a format that is hard to work with. Let's start by creating some month columns.

In [26]:
#create month columns
df['Month_Account_Created'] = df['date_account_created'].dt.month
df['Month_First_Active'] = df['timestamp_first_active'].dt.month
# note -- other things could be tried here. Maybe people who book at 4am choose different
#destinations than those who book at 1pm?, etc.

In [27]:
df.head()

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,entered_age,Month_Account_Created,Month_First_Active
0,gxn3p5htnn,2010-06-28,2009-03-19 04:32:55,-unknown-,34,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF,0,6,3
1,820tgsjxq7,2011-05-25,2009-05-23 17:48:09,MALE,38,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF,1,5,5
2,4ft3gnwmtx,2010-09-28,2009-06-09 23:12:47,FEMALE,56,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US,1,9,6
3,bjjt8pjhuk,2011-12-05,2009-10-31 06:01:29,FEMALE,42,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other,1,12,10
4,87mebub9p4,2010-09-14,2009-12-08 06:11:05,-unknown-,41,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US,1,9,12


Fill categorical NaNs with 'nothing'.

In [28]:
df['first_affiliate_tracked'].fillna('nothing', inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 213451 entries, 0 to 213450
Data columns (total 18 columns):
id                         213451 non-null object
date_account_created       213451 non-null datetime64[ns]
timestamp_first_active     213451 non-null datetime64[ns]
gender                     213451 non-null object
age                        213451 non-null float64
signup_method              213451 non-null object
signup_flow                213451 non-null int64
language                   213451 non-null object
affiliate_channel          213451 non-null object
affiliate_provider         213451 non-null object
first_affiliate_tracked    213451 non-null object
signup_app                 213451 non-null object
first_device_type          213451 non-null object
first_browser              213451 non-null object
country_destination        213451 non-null object
entered_age                213451 non-null int64
Month_Account_Created      213451 non-null int64
Month_First_Active       

Bin together affiliate_channel values of 'api', 'content', and 'remarketing' as 'binned_other'.

In [29]:
df.affiliate_channel.value_counts()

direct           137727
sem-brand         26045
sem-non-brand     18844
other              8961
seo                8663
api                8167
content            3948
remarketing        1096
Name: affiliate_channel, dtype: int64

In [30]:
# NEW 2/1 have content be its own bucket because it behaves differently

channels_to_bin = ['api', 'remarketing']

#change values to binned_other
merged2.loc[merged2.affiliate_channel.isin(channels_to_bin), 'affiliate_channel'] = 'binned_other'

# check        
merged2['affiliate_channel'].value_counts()

direct           137727
sem-brand         26045
sem-non-brand     18844
binned_other       9263
other              8961
seo                8663
content            3948
Name: affiliate_channel, dtype: int64

In [31]:
merged2.signup_method.value_counts()

basic       152897
facebook     60008
google         546
Name: signup_method, dtype: int64

In [32]:
merged2.signup_flow.value_counts()

0     164739
25     14659
12      9329
3       8822
2       6881
24      4328
23      2835
1       1047
6        301
8        240
21       196
5         36
20        14
16        11
15        10
10         2
4          1
Name: signup_flow, dtype: int64

In [33]:
# top 5 cover 95% of my data
# bin the rest as other
#could also come back and treat as numerical. Maybe the higher numbers are just signps that are lower down the funnel?
# 2/1 very few of these are represented in test data.

# NEW 2/1
buckets_to_keep = [0,25,23,12,8]
merged2.loc[~merged2.signup_flow.isin(buckets_to_keep), 'signup_flow'] = -1

In [34]:
merged2.signup_flow.value_counts()

 0     164739
-1      21649
 25     14659
 12      9329
 23      2835
 8        240
Name: signup_flow, dtype: int64

In [35]:
merged2.language.value_counts()

en    206314
zh      1632
fr      1172
es       915
ko       747
de       732
it       514
ru       389
pt       240
ja       225
       ...  
cs        32
no        30
el        24
th        24
id        22
hu        18
fi        14
is         5
ca         5
hr         2
Name: language, dtype: int64

In [36]:
# group together non-english languages as 'other'
merged2.loc[merged2['language'] != 'en','language'] = 'other'
    
# check        
merged2['language'].value_counts()

en       206314
other      7137
Name: language, dtype: int64

In [37]:
merged2.affiliate_provider.value_counts()

direct                 137426
google                  51693
other                   12549
craigslist               3471
bing                     2328
facebook                 2273
vast                      829
padmapper                 768
facebook-open-graph       545
yahoo                     496
gsp                       453
meetup                    347
email-marketing           166
naver                      52
baidu                      29
yandex                     17
wayn                        8
daum                        1
Name: affiliate_provider, dtype: int64

In [38]:
#affiliate providers to keep
aff_provider_to_keep = ['direct', 'google', 'craigslist']

# change the other affiliate provider values to other
merged2.loc[~merged2.affiliate_provider.isin(aff_provider_to_keep),'affiliate_provider'] = 'other'
        
# check        
merged2['affiliate_provider'].value_counts()

direct        137426
google         51693
other          20861
craigslist      3471
Name: affiliate_provider, dtype: int64

In [39]:
merged2.first_affiliate_tracked.value_counts()

untracked        109232
linked            46287
omg               43982
tracked-other      6156
product            1556
marketing           139
local ops            34
Name: first_affiliate_tracked, dtype: int64

In [40]:
# affiliates tracked to keep. added local_ops because it behaves differently.
aff_tracked_to_keep = ['linked', 'omg', 'local ops']

# changed remaining values to other
merged2.loc[~merged2.first_affiliate_tracked.isin(aff_tracked_to_keep),'first_affiliate_tracked'] = 'other'
       
# check        
merged2['first_affiliate_tracked'].value_counts()

other        123148
linked        46287
omg           43982
local ops        34
Name: first_affiliate_tracked, dtype: int64

In [41]:
merged2.signup_app.value_counts()

Web        182717
iOS         19019
Moweb        6261
Android      5454
Name: signup_app, dtype: int64

In [42]:
# NEW 2/1
# smartphone (other) grouped with iPhone because of similar behavior
merged2.first_device_type.value_counts()

Mac Desktop           89600
Windows Desktop       72716
iPhone                20759
iPad                  14339
Other/Unknown         10667
Android Phone          2803
Android Tablet         1292
Desktop (Other)        1199
SmartPhone (Other)       76
Name: first_device_type, dtype: int64

In [43]:
#for i in range(0, len(merged2['first_device_type'])):
#    if merged2.ix[i,'first_device_type'] == 'Mac Desktop' or merged2.ix[i,'first_device_type'] == 'Windows Desktop'or merged2.ix[i,'first_device_type'] == 'Desktop (Other)':
#        merged2.ix[i,'first_device_type'] = 'desktop'
#    elif merged2.ix[i,'first_device_type'] == 'iPhone' or merged2.ix[i,'first_device_type'] == 'Android Phone' or merged2.ix[i,'first_device_type'] == 'SmartPhone (Other)':
#        merged2.ix[i,'first_device_type'] = 'phone'
#    else merged2.ix[i,'first_device_type'] == 'iPad' or merged2.ix[i,'first_device_type'] == 'Android Tablet':
#        merged2.ix[i,'first_device_type'] = 'tablet'

buckets_to_merge = ['SmartPhone (Other)', 'iPhone']
merged2.loc[merged2.first_device_type.isin(buckets_to_merge), 'first_device_type'] = 'iPhone/Other Smartphone'

# check        
merged2['first_device_type'].value_counts()

Mac Desktop                89600
Windows Desktop            72716
iPhone/Other Smartphone    20835
iPad                       14339
Other/Unknown              10667
Android Phone               2803
Android Tablet              1292
Desktop (Other)             1199
Name: first_device_type, dtype: int64

In [44]:
merged2.first_browser.value_counts()

Chrome                  63845
Safari                  45169
Firefox                 33655
-unknown-               27266
IE                      21068
Mobile Safari           19274
Chrome Mobile            1270
Android Browser           851
AOL Explorer              245
Opera                     188
                        ...  
IceDragon                   1
Stainless                   1
Conkeror                    1
Outlook 2007                1
Palm Pre web browser        1
Kindle Browser              1
Epic                        1
Google Earth                1
Arora                       1
NetNewsWire                 1
Name: first_browser, dtype: int64

In [45]:
# NEW 2/1 new bucketing

browsers_to_keep = ['Chrome','Safari', 'Firefox', 'IE', 'Mobile Safari', 'Chrome Mobile']
merged2.loc[~merged2.first_browser.isin(browsers_to_keep), 'first_browser'] = 'Other'

# check        
merged2['first_browser'].value_counts()

Chrome           63845
Safari           45169
Firefox          33655
Other            29170
IE               21068
Mobile Safari    19274
Chrome Mobile     1270
Name: first_browser, dtype: int64

In [ ]:
# NEXT: scaling data with zero mean and unit variance